In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.options.display.max_rows = 75
import plotly.express as px

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
df = pd.read_csv('/kaggle/input/olympic-games-hosts/olympic_hosts.csv')
df.head()

In [ ]:
df_country_iso = pd.read_csv('https://raw.githubusercontent.com/lukes/ISO-3166-Countries-with-Regional-Codes/master/all/all.csv')
df_country_iso

In [ ]:
country_iso_dict = {
    "USSR": "Russian Federation",
    "Yugoslavia": "Bosnia and Herzegovina", 
    "Great Britain": "United Kingdom of Great Britain and Northern Ireland",
    "Republic of Korea": "Korea (Democratic People's Republic of)",
    "People's Republic of China": "China"
}
iso_country_dit = {
    "United States of America": "USA",
    "United Kingdom of Great Britain and Northern Ireland": "Great Britain",
    "Korea (Democratic People's Republic of)": "Korea",
    "Russian Federation": "Russia"
    }
alpha_3_dict = dict(zip(df_country_iso['name'], df_country_iso['alpha-3']))

In [ ]:
## Melt Country column
melt_column = 'Country'

df_country = df[melt_column].apply(lambda x: x.split(', ')).apply(pd.Series) \
                            .merge(df, left_index = True, right_index = True) \
                            .drop([melt_column], axis=1) \
                            .melt(id_vars = [item for item in df.columns.tolist() if item!=melt_column], 
                                  value_vars = [0,1], value_name=melt_column) \
                            .drop("variable", axis=1) \
                            .dropna(subset=[melt_column]) \
                            .sort_values('Year').reset_index(drop=True)

In [ ]:
df_country['CountryISO'] = df_country['Country'].replace(country_iso_dict)
df_country['Country-alpha-3'] = df_country['CountryISO'].map(alpha_3_dict)

In [ ]:
df_map = df_country.groupby('CountryISO').size().reset_index().rename(columns={0:'Hosts'})
df_map['CountryCode'] = df_map['CountryISO'].map(alpha_3_dict)
df_map['CountryISO'] = df_map['CountryISO'].replace(iso_country_dit)
# df_map.sort_values('Hosts', ascending=False)

In [ ]:
font=dict(
    family="Courier New, monospace",
    size=16,
    color="#7f7f7f"
)

In [ ]:
fig = px.choropleth(df_map, locations="CountryCode",
                    color="Hosts",
                    hover_name="CountryISO",
                    color_continuous_scale=px.colors.sequential.Plasma)
fig.update_layout(
    title="Olympic Games Map",
    font=font
)
fig.show()

In [ ]:
fig = px.bar(df[df['Type']=='summergames'], x='Year', y='Athletes', text='Country')
fig.update_traces(texttemplate='%{x:.3s}', textposition='outside')
fig.update_layout(
    title="Number Athlets, Summer Games",
    xaxis_title="Year",
    yaxis_title="",
    font=font,
    uniformtext_minsize=8, 
    uniformtext_mode='hide')

fig.show()

In [ ]:
# # fig = px.bar(df[df['Type']=='summergames'], y='Year', x='Athletes', text='Country', orientation='h')

# fig = px.line(df, x="Year", y="Athletes", color="Type")

# # fig.update_traces(texttemplate='%{x:.3s}', textposition='outside')

# fig.update_layout(
#     title="Number Athlets, Summer Games",
#     xaxis_title="Year",
#     yaxis_title="",
#     font=font,
#     width=2000,
#     height=1000,
#     uniformtext_minsize=8, 
#     uniformtext_mode='hide')

# fig.show()